When you run a notebook cell that imports from the whendo packages, you need to set the NOTEBOOK FILE ROOT to \$\{workspaceFolder\} so  that imports from top-level packages work. For example, with the property's default value of \$\{fileDirName\}, the imports from dispatcher.etc... will fail since the notebook location is one directory down from top-level.





In [ ]:
from datetime import time, datetime, timedelta
from pydantic import BaseModel
import requests
import json
from whendo.core.action import Action
from whendo.core.dispatcher import Dispatcher
import whendo.core.schedulers.cont_scheduler as sched_x
import whendo.core.actions.file_action as file_x
import whendo.core.actions.logic_action as logic_x
import whendo.core.actions.sys_action as sys_x
from whendo.core.program import Program
from whendo.core.util import PP, TimeUnit, Dirs, DateTime, Now, DateTime2
from whendo.core.resolver import resolve_action

prefix = "http://192.168.0.45:8000"
prefix = "http://127.0.0.1:8000"
prefix = "http://192.168.0.26:8000"
def get(path:str):
    cmd = f"{prefix}{path}"
    response = requests.get(cmd)
    return response.json()
def put(path:str, data:BaseModel):
    cmd = f"{prefix}{path}"
    response = requests.put(cmd, data.json())
    PP.pprint(response.json())
def post(path:str, data:BaseModel):
    cmd = f"{prefix}{path}"
    response = requests.post(cmd, data.json())
    PP.pprint(response.json())
def post_dict(path:str, data:dict):
    cmd = f"{prefix}{path}"
    response = requests.post(cmd, json.dumps(data))
    PP.pprint(response.json())
def delete(path:str):
    cmd = f"{prefix}{path}"
    response = requests.delete(cmd)
    return response.json()

In [ ]:
get('/dispatcher/clear')

In [ ]:
get('/dispatcher/load')

In [ ]:
get('/dispatcher/describe_all')

In [ ]:
get('/dispatcher/clear')

heart_1 = file_x.FileHeartbeat(file="heartbeat1.txt", payload={'words':'heartbreak hotel'})
post('/actions/heartbeat1', heart_1)
heart_2 = file_x.FileHeartbeat(file="heartbeat2.txt", payload={'words':'nothing but heartaches'})
post('/actions/heartbeat2', heart_2)
heart_3 = file_x.FileHeartbeat(file="heartbeat3.txt", payload={'words':'heart of glass'})
post('/actions/heartbeat3', heart_3)

append_1 = file_x.FileAppend(file="append_1.txt")
post('/actions/append_1', append_1)
append_2 = file_x.FileAppend(file="append_2.txt")
post('/actions/append_2', append_2)
append_3 = file_x.FileAppend(file="append_3.txt", payload={'words':'apply'})
post('/actions/append_3', append_3)

sys_info = sys_x.SysInfo()
post('/actions/sys_info', sys_info)
pause = sys_x.Pause()
post('/actions/pause', pause)
logic_1 = logic_x.All(actions=[heart_1, heart_2])
post('/actions/logic1', logic_1)
success = logic_x.Success()
post('/actions/success', success)
file_append = file_x.FileAppend(file="boomerang.txt")
post('/actions/file_append', file_append)
mini_info = sys_x.MiniInfo()
post('/actions/mini_info', mini_info)

info_append_1 = logic_x.All(actions=[sys_info, append_1])
post('/actions/info_append_1', info_append_1)
info_append_2 = logic_x.All(actions=[sys_info, append_2])
post('/actions/info_append_2', info_append_2)
info_append_3 = logic_x.All(actions=[mini_info, append_3])
post('/actions/info_append_3', info_append_3)

scheduler = sched_x.Randomly(time_unit=TimeUnit.second, low=5, high=10)
post('/schedulers/randomly_soon', scheduler)
scheduler = sched_x.Timely(interval=1)
post('/schedulers/often', scheduler)
morning, evening = time(6,0,0), time(18,0,0)
scheduler = sched_x.Timely(interval=1, start=morning, stop=evening)
post('/schedulers/timely_day', scheduler)
scheduler = sched_x.Timely(interval=1, start=evening, stop=morning)
post('/schedulers/timely_night', scheduler)
scheduler = sched_x.Immediately()
post('/schedulers/immediately', scheduler)

program = Program().prologue("heartbeat1").epilogue("heartbeat3").body_element("often", "heartbeat2")
post('/programs/program1', program)
info_append = Program().prologue("info_append_1").epilogue("info_append_3").body_element("often", "info_append_2")
post('/programs/info_append', info_append)

if True:
    start = Now.dt()+timedelta(seconds=2)
    stop = start + timedelta(seconds=20)
    datetime2 = DateTime2(dt1=start, dt2=stop)
    post("/programs/info_append/schedule", datetime2)
elif True:
    start = Now.dt()
    stop = start + timedelta(seconds=20)
    datetime2 = DateTime2(dt1=start, dt2=stop)
    post("/programs/program1/schedule", datetime2)
elif True:
    get('/schedulers/often/actions/logic1')
    dt = DateTime(date_time=Now.dt()+timedelta(seconds=10))
    post('/schedulers/often/actions/logic1/expire', dt)
    post('/schedulers/often/actions/heartbeat3/defer', dt)
elif True: # write once to heartbeat1 & heartbeat2
    dt = DateTime(date_time=Now.dt()+timedelta(seconds=10))
    post('/schedulers/immediately/actions/logic1/defer', dt)

# get('/schedulers/timely_day/actions/logic1')
# get('/schedulers/timely_night/actions/logic1')
# get('/schedulers/timely_day/actions/heartbeat1')
# get('/schedulers/timely_night/actions/heartbeat2')
# get('/schedulers/randomly_soon/actions/heartbeat3')

In [ ]:
get('/jobs/run')

In [ ]:
get('/jobs/stop')

In [ ]:
get('/jobs/count')

In [ ]:
get('/actions/sys_info/execute')